In [63]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.image as mpimg
import cv2

In [49]:
im_shape = 350, 470, 3
person_num = 40

X = tf.placeholder(tf.float32, [None, im_shape[0], im_shape[1], im_shape[2]])
Y = tf.placeholder(tf.float32, [None, person_num])
keep_prob = tf.placeholder(tf.float32)
global_step = tf.Variable(0, trainable=False, name="global_step")

In [51]:
# 3x3 크기의 한칸씩 움직이는 커널을 32개 만든다.
W1 = tf.Variable(tf.random_normal([3,3,3,32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1,1,1,1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 4, 4, 1], strides=[1, 4, 4, 1], padding='SAME')

L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3, 512, activation=tf.nn.relu)
L3 = tf.layers.dropout(L3, keep_prob)

L4 = tf.contrib.layers.flatten(L3)
L4 = tf.layers.dense(L4, 256, activation=tf.nn.relu)
L4 = tf.layers.dropout(L4, keep_prob)

feature_vec = L4

logits_M = tf.layers.dense(L4, person_num, activation=None, trainable=False)
cost_M = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_M, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost_M)

In [59]:
files = [f for f in os.listdir('./data/')]

In [79]:
a = cv2.imread('./data/' + files[1])

In [67]:
files[0]

'cut_U10_10_0_18.png'

In [77]:
b = np.eye(person_num)[9].reshape([1,40])
c = np.eye(2)[1]

In [78]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

sess.run([optimizer, cost_M], feed_dict={X: a, Y: b})

[None, 5.0170307]